FHP6QZ-Numerical_Analysis_Assignment-04

In [69]:
#Importing required Libraries
import matplotlib.pyplot as plt
from IPython.display import  Latex
from IPython.display import display
from IPython.display import Math

import numpy as np
np.set_printoptions(precision=4)



In [70]:
#Function to define A Matrix
def Get_A(n):
    A = np.zeros([n, n])
    A[0, :2] = [2, -1]
    
    for j in range(1, n-1):
        A[j, j-1:j+2] = [-1, 2, -1]
        
    A[n-1, n-2:n] = [-1, 2]
    A = (n + 1)**2 * A
    return A

 Exact Value λk = 4(n + 1)2
sin2
(
kπ
2(n+1) )

In [71]:
#Defining Lambda
def Get_Lambdas(n):
    lambdas_list = list()
    

    for i in range(1, n + 1):
        lambdas_list.append(4 * (n + 1)**2 * np.sin((i * np.pi) / (2 * (n + 1)))**2)
        
    return lambdas_list

In [72]:
# Function defining QR_Algorithm
def QR_Algo(A):
    Q = np.eye(A.shape[0])
    R = A
    for k in range(A.shape[1] - 1):
        x = R[k:, k]
        sign_x1 = x[0] / abs(x[0]) if x[0] != 0 else 1
        E1 = np.zeros_like(x)
        E1[0] = 1
        v = sign_x1 * np.linalg.norm(x) * E1 + x
        G = np.eye(len(x)) - 2 / np.dot(v, v.T) * np.outer(v, v)
        
        Q_i = np.eye(A.shape[0])
        Q_i[-G.shape[0]:, -G.shape[1]:] = G
        
        R = Q_i @ R
        Q = Q @ Q_i
    
    return Q, R

In [73]:
# Funtion for Tridiagonal Matrix
def Tridiagonal_Mat(N, p): #
    
    a = np.array([N[i, i-1] for i in range(1, N.shape[0])])
    b = np.array([N[i, i] for i in range(N.shape[0])])
    c = np.array([N[i, i+1] for i in range(N.shape[0] - 1)])
    
    u = [b[0]]
    v = [p[0]]

    for j in range(1, len(p)):
        l = a[j-1] / u[j-1]
        u.append(b[j] - l * c[j-1])
        v.append(p[j] - l * v[j-1])
        
    X = np.zeros(len(p))
    X[-1] = v[-1] / u[-1]
    
    for k in range(len(X) - 2, -1, -1):
        assert X[k] == 0
        X[k] = (v[k] - c[k] * X[k+1]) / u[k]

    #print(X)   
    return X

In [74]:
# Function to calculate the Eigen Values
def Get_Eigen_values(A, epsilion=0.01):

    eigen_values = list()
    
    while A.shape[0] > 1:
        H = A
        prev_d = 0
        while abs(A[-1, -2]) > epsilion:
            if prev_d == np.round(A[-1, -2], 4):
                # Perturbation if needed
                M = A[-1, -1] + 0.2
            else:
                M = A[-1, -1]
                
            prev_d = np.round(A[-1, -2], 4)
            
            Q, R = QR_Algo(A - M * np.eye(A.shape[0]))
            A = R @ Q + M * np.eye(A.shape[0])
            
        eigen_values.append(A[-1, -1])
        A = A[:-1, :-1]
        
    eigen_values.append(A[-1, -1])
    
    return eigen_values        

A). For N=10, Calculating and Comparing the Eigen Values.

In [75]:
#Matrix A
A = Get_A(10)
print(A)

[[ 242. -121.    0.    0.    0.    0.    0.    0.    0.    0.]
 [-121.  242. -121.    0.    0.    0.    0.    0.    0.    0.]
 [   0. -121.  242. -121.    0.    0.    0.    0.    0.    0.]
 [   0.    0. -121.  242. -121.    0.    0.    0.    0.    0.]
 [   0.    0.    0. -121.  242. -121.    0.    0.    0.    0.]
 [   0.    0.    0.    0. -121.  242. -121.    0.    0.    0.]
 [   0.    0.    0.    0.    0. -121.  242. -121.    0.    0.]
 [   0.    0.    0.    0.    0.    0. -121.  242. -121.    0.]
 [   0.    0.    0.    0.    0.    0.    0. -121.  242. -121.]
 [   0.    0.    0.    0.    0.    0.    0.    0. -121.  242.]]


In [76]:
#Calculating the Eigen Values of A.
#Get_Eigen_values(A)
sorted(Get_Eigen_values(A))

[9.802700823417684,
 38.41664461675475,
 83.52370238999937,
 141.46956699162266,
 207.5598091380578,
 276.440190862135,
 342.53043314625927,
 400.4762974746387,
 445.58335494513585,
 474.1972996119794]

In [77]:
# Values from the Lambda function.
#Eigen values similar to the Get_Eigen_values(A)
Get_Lambdas(10)

[9.802700385291631,
 38.41664505485415,
 83.523702385241,
 141.46956685354348,
 207.55980913786502,
 276.440190862135,
 342.53043314645646,
 400.47629761475895,
 445.5833549451458,
 474.19729961470836]

In [78]:
# Eigen_values from Builtin numpy
eigen_value, _ = np.linalg.eig(A)  
sorted(eigen_value)

[9.802700385291486,
 38.416645054854094,
 83.52370238524095,
 141.46956685354348,
 207.55980913786493,
 276.4401908621345,
 342.5304331464561,
 400.476297614759,
 445.5833549451454,
 474.19729961470756]

B). Inverse iteration to find eigenvector corresponding to the smallest
eigenvalue of A.

In [79]:
#Function to calculate Eigen Vectors
def Get_Eigen_Vector(n):
    eigen_vector = list()
    
    for j in range(1, n + 1):
        eigen_vector.append(np.sin((np.pi * j) / (n + 1)))
        
    eigen_vector = np.array(eigen_vector)
    #Normalizing the values
    eigen_vector = eigen_vector / np.linalg.norm(eigen_vector)
    
    return eigen_vector

In [80]:
#Defining Inverse Iteration Function
def Inverse_Iteration(A, theta, epsilion=0.001):
    prev_v = np.zeros(A.shape[0])
    V = np.ones(A.shape[0]) / np.linalg.norm(np.ones(A.shape[0]))
    
    while np.linalg.norm(abs(V) - abs(prev_v)) > epsilion:
        prev_v = V
        u = Tridiagonal_Mat(A - theta * np.eye(len(V)), V)
        V = u / np.linalg.norm(u)
        
        error = np.linalg.norm(u - np.dot(prev_v, u) * prev_v)
        
    return V

In [81]:
smallest_eigen_value = min(Get_Eigen_values(A))
smallest_eigen_value

9.802700823417684

In [82]:
#Calculating eigenvector corresponding to the smallest eigenvalue of A
print(Inverse_Iteration(A, smallest_eigen_value))  

[0.1201 0.2305 0.3223 0.3879 0.4221 0.4221 0.3879 0.3223 0.2305 0.1201]


In [83]:
# Eigen vector using the defined Function
print(Get_Eigen_Vector(10))  

[0.1201 0.2305 0.3223 0.3879 0.4221 0.4221 0.3879 0.3223 0.2305 0.1201]


In [84]:
# Smallest eigen value and corresponding eigen vectors from built-in numpy
x, y = np.linalg.eig(A)  
min_idx = np.argmin(x)
print(x[min_idx])
print(y[:, min_idx])


9.802700385291486
[-0.1201 -0.2305 -0.3223 -0.3879 -0.4221 -0.4221 -0.3879 -0.3223 -0.2305
 -0.1201]
